In [1]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from utils import call_openai
from pydantic import BaseModel, ValidationError
from typing import List
import json

class FAQItem(BaseModel):
    index: int
    question: str
    answer: str

class FAQList(BaseModel):
    faqs: List[FAQItem]

# 나무위키 대학수학능력시험 관련 페이지 URL
url = "https://namu.wiki/w/%EB%8C%80%ED%95%99%EC%88%98%ED%95%99%EB%8A%A5%EB%A0%A5%EC%8B%9C%ED%97%98"

loader = AsyncHtmlLoader([url],header_template={"User-Agent": "WangHS"})

docs = loader.load()

content = docs[0].page_content

transformer = Html2TextTransformer()

plain = transformer.transform_documents(docs)

plain_text = plain[0].page_content

prompt = """대표적인 10의 FAQ를 아래의 context를 보고 생성하시오.
출력포맷은 리스트이며, 세부 내용은 다음과 같습니다.
반드시 출력포맷만을 생성하고, 다른 텍스트는 생성하지 마시오.
[{"index": <인덱스 번호>, "question": <질문>, "answer": <답변>}, ...]

Context:
"""

FAQ = call_openai(prompt + plain_text, model='gpt-4o-2024-05-13')

FAQ_data = json.loads(FAQ)
# Pydantic 모델로 데이터 구조화
try:
    structured_faq = FAQList(faqs=FAQ_data)  # FAQ 데이터를 Pydantic 모델로 변환
    print("구조화된 FAQ 데이터:\n")
    for item in structured_faq.faqs:
        print(f"Index: {item.index}, Question: {item.question}, Answer: {item.answer}")
except ValidationError as e:
    print(f"검증 오류 발생: {e}")


USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.06it/s]


구조화된 FAQ 데이터:

Index: 1, Question: 대학수학능력시험이란 무엇인가요?, Answer: 대학수학능력시험은 고등학교 졸업(예정)자를 대상으로 대학 교육에 필요한 수학 능력을 측정하기 위해 한국교육과정평가원에서 매년 11월 셋째 토요일 직전 목요일마다 시행하는 표준화 시험입니다.
Index: 2, Question: 대학수학능력시험의 공식 명칭은 무엇인가요?, Answer: 공식 명칭은 '대학수학능력시험(大學修學能力試驗)'이며, 보통 이를 줄여 '수능(修能)', '대수능(大修能)'이라고 부릅니다. 영어 명칭은 'College Scholastic Ability Test'이며, 두문자어로 'CSAT'이라고 합니다.
Index: 3, Question: 대학수학능력시험의 역사에 대해 알려주세요., Answer: 대학수학능력시험은 1993년부터 시행되었으며, 그 이전에는 대학입학 예비고사와 대학입학 학력고사가 있었습니다. 수능은 기존의 학력고사 체계와의 차별성을 두고 평가 기준과 목적, 평가 방법을 개혁한 시험입니다.
Index: 4, Question: 대학수학능력시험의 주요 영역은 무엇인가요?, Answer: 대학수학능력시험의 주요 영역은 국어, 수학, 영어, 한국사, 사회탐구, 과학탐구, 직업탐구, 제2외국어/한문으로 나뉩니다.
Index: 5, Question: 대학수학능력시험의 성적 산출 방식은 어떻게 되나요?, Answer: 대학수학능력시험의 성적은 표준점수, 백분위, 등급으로 산출됩니다. 표준점수는 원점수를 기반으로 평균과 표준편차를 이용해 산출되며, 백분위는 응시자 수를 100명으로 가정했을 때의 상대적 위치를 나타냅니다. 등급은 백분위에 따라 1~9등급으로 나뉩니다.
Index: 6, Question: 대학수학능력시험의 출제 과정은 어떻게 되나요?, Answer: 대학수학능력시험의 출제는 대학 교수와 고등학교 교사로 구성된 출제위원과 검토위원이 참여하며, 출제 시작부터 시험 종료까지 철저한 보안 속에서 진행됩니다. 문제 출제 후에는 여러

In [ ]:
from utils import get_embeddings, get_embedding
import faiss
import numpy as np

questions = [item.question for item in structured_faq.faqs]

embqs = np.array(get_embeddings(questions))

index = faiss.IndexIDMap(faiss.IndexFlatIP(embqs.shape[1]))
ids = np.array([item.index for item in structured_faq.faqs])
index.add_with_ids(embqs, ids)

faiss.write_index(index, './res/exam_FAQ.index')


검색 결과:
1: 부정행위 유형에는 다른 수험생의 답안지를 보거나 자신의 답안지를 보여주는 행위, 부정한 휴대물을 보거나 무선기기 등을 이용하는 행위 등이 있으며, 적발 시 당해 시험 무효 및 1년간 응시 자격 정지 등의 처벌을 받습니다.
2: 대학수학능력시험의 출제는 대학 교수와 고등학교 교사로 구성된 출제위원과 검토위원이 참여하며, 출제 시작부터 시험 종료까지 철저한 보안 속에서 진행됩니다. 문제 출제 후에는 여러 차례 검토와 수정 과정을 거쳐 최종 확정됩니다.
3: 대학수학능력시험의 성적은 시험일로부터 약 20일 후에 통지되며, 표준점수, 백분위, 등급이 포함된 성적표가 제공됩니다. 성적표는 온라인으로 확인할 수 있으며, 필요 시 종이 성적표도 발급받을 수 있습니다.
4: 대학수학능력시험의 주요 영역은 국어, 수학, 영어, 한국사, 사회탐구, 과학탐구, 직업탐구, 제2외국어/한문으로 나뉩니다.
5: 대학수학능력시험은 고등학교 졸업(예정)자를 대상으로 대학 교육에 필요한 수학 능력을 측정하기 위해 한국교육과정평가원에서 매년 11월 셋째 토요일 직전 목요일마다 시행하는 표준화 시험입니다.


In [ ]:
import pickle

with open('./res/faq_data.pkl', 'wb') as f:
    pickle.dump(structured_faq, f)


In [25]:
# ID를 키로 FAQItem 객체를 저장하는 딕셔너리 생성
faq_dict = {item.index: item for item in structured_faq.faqs}

def search(query):
    # 검색할 쿼리의 임베딩 생성
    query_vector = np.array(get_embedding(query), dtype=np.float32).reshape(1, -1)
    
    # 상위 K개의 결과 검색
    k = 3
    distances, indices = index.search(query_vector, k)
    
    # ID를 통해 structured_faq.faqs에서 question과 answer 추출
    results = [(faq_dict[_id].question, faq_dict[_id].answer) for _id in indices[0] if _id != -1]
    
    return results

# 예시 쿼리 실행
query = "대학수학능력시험은 언제 생겼나요?"
results = search(query)

# 검색 결과 출력
print("검색 결과:")
for idx, (question, answer) in enumerate(results, 1):
    print(f"{idx}. 질문: {question}")
    print(f"   답변: {answer}")



prompt_faq = f"""제시된 관련있는 FAQ를 참고하여 아래의 질문에 적절한 대답을 하시오.
만약 제시된 FAQ와 질문이 관련성이 없다면 관련된 FAQ가 존재하지 않는다고 답하시오.

FAQ:
{results}
Question:
{query}
"""

final_result = call_openai(prompt_faq, model='gpt-4o-mini')
print(final_result)

검색 결과:
1. 질문: 대학수학능력시험의 역사에 대해 알려주세요.
   답변: 대학수학능력시험은 1993년부터 시행되었으며, 그 이전에는 대학입학 예비고사와 대학입학 학력고사가 있었습니다. 수능은 기존의 학력고사 체계와의 차별성을 두고 평가 기준과 목적, 평가 방법을 개혁한 시험입니다.
2. 질문: 대학수학능력시험의 출제 과정은 어떻게 되나요?
   답변: 대학수학능력시험의 출제는 대학 교수와 고등학교 교사로 구성된 출제위원과 검토위원이 참여하며, 출제 시작부터 시험 종료까지 철저한 보안 속에서 진행됩니다. 문제 출제 후에는 여러 차례 검토와 수정 과정을 거쳐 최종 확정됩니다.
3. 질문: 대학수학능력시험의 공식 명칭은 무엇인가요?
   답변: 공식 명칭은 '대학수학능력시험(大學修學能力試驗)'이며, 보통 이를 줄여 '수능(修能)', '대수능(大修能)'이라고 부릅니다. 영어 명칭은 'College Scholastic Ability Test'이며, 두문자어로 'CSAT'이라고 합니다.
대학수학능력시험은 1993년부터 시행되었습니다.
